<a href="https://colab.research.google.com/github/xuwangfmc/dlbook/blob/main/wb_hydra/Sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sweep

该教程主要介绍如何将Weights&Bias的Artifacts工具运用到PyTorch当中，实现模型的超参数搜索以找到性能表现最好的模型设置。

Weights&Bias的Artifacts页面如下所示：
![6P0ATeH.png](https://s2.loli.net/2022/01/22/57duZjnVeXcQv1m.png)

Sweep记录了每次不同超参数设置下的模型表现，以及各超参数对模型性能的重要性。

运行Weights&Bias的超参数搜索方法主要分为三个步骤：

- 1) 定义Sweep的配置：可以通过创建字典或者YAML文件来设置需要搜索的超参数、搜索策略、优化指标等。

- 2) 初始化Sweep：通过wandb.sweep(sweep_config)传入Sweep的配置和完成Sweep的初始化。

- 3) 运行Sweep代理：调用wandb.agent()函数，传入sweep_id以及定义了模型结构的训练函数:wandb.agent(sweep_id, function=train)

## 安装wandb库并登录账号

在https://wandb.ai/authorize 登录账号并粘贴API key。

In [1]:
!pip install wandb
!wandb login

     |████████████████████████████████| 1.7 MB 13.1 MB/s 
     |████████████████████████████████| 142 kB 70.5 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 180 kB 66.9 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=55f58c94f9765dddff51f098f215195c9024b71af85991baad02334087f7f281
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1db381a2083d24e21687e787cf86f23e17931fef51afc594740b1636c512e95a
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or 

## 定义Sweep的配置

通过创建字典或者YAML文件可以完成Sweep的配置。

- Metric：这是Sweep尝试优化的指标。Metric有name和goal两个参数。

- 搜索策略：运用'method'变量指定。常用有随机搜索、网格搜索、贝叶斯搜索三种方式。

- Parameters：包含一系列超参数的字典，可以是离散值、最大值、最小值或者某种分布。

In [2]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torch.nn as nn
import wandb
from torchvision import datasets, transforms

sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [2, 5, 10]
        },
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0.3, 0.4, 0.5]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        'fc_layer_size':{
            'values':[128,256,512]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
    }
}

## 初始化Sweep

In [4]:
sweep_id = wandb.sweep(sweep_config, project="Pytorch-sweeps")

Create sweep with ID: ea46d7yc
Sweep URL: https://wandb.ai/xuwangfmc/Pytorch-sweeps/sweeps/ea46d7yc


配置数据集

In [5]:
def build_dataset(batch_size):
   transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
   dataset = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
   train_loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size)

   return train_loader

定义网络架构及训练方式如下：

In [7]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 5,
        'batch_size': 128,
        'learning_rate': 1e-3,
        'optimizer': 'adam',
        'fc_layer_size': 128,
        'dropout': 0.5,
    }
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
    # Define the model architecture - This is a simplified version of the VGG19 architecture
    network = nn.Sequential(
     nn.Flatten(start_dim=1)
    ,nn.Linear(784, config.fc_layer_size)
    ,nn.ReLU()
    ,nn.Dropout(config.dropout)
    ,nn.Linear(config.fc_layer_size, 10)
    ,nn.LogSoftmax(dim=1)
     )
    train_loader = build_dataset(config.batch_size)
    # Set of Conv2D, Conv2D, MaxPooling2D layers with 32 and 64 filters

    # Define the optimizer
    if config.optimizer=='sgd':
      optimizer = optim.SGD(network.parameters(),lr=config.learning_rate, momentum=0.9)
    elif config.optimizer=='adam':
      optimizer = optim.Adam(network.parameters(),lr=config.learning_rate)

    network.train()
    network = network.to(device)
    for i in range(config.epochs):
      closs= 0
      for batch_idx, (data, target) in enumerate(train_loader):
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = network(data)
          loss = F.nll_loss(output, target)
          loss.backward()
          closs = closs + loss.item()
          optimizer.step()
          wandb.log({"batch loss":loss.item()})
      wandb.log({"loss":closs/config.batch_size}) 

## 运行Sweep代理

In [8]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 136ep5ty with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: Currently logged in as: xuwangfmc (use `wandb login --relogin` to force relogin)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



...

batch loss,█▄▂▃▃▃▂▂▃▂▂▂▂▂▃▂▂▂▂▁▂▂▁▁▂▂▂▁▁▁▂▁▁▂▂▂▂▂▂▃
loss,█▁
batch loss,0.28656
loss,0.78056


wandb: Agent Starting Run: 9wnygjzv with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam


batch loss,▆▃▅▅▂▃▄▃▆▆▄▄▄▆▆▃▆▂▃▂▄█▁▃▄▇▃▇▃▄▆▃▄▂▃▂▄▆▃▆
loss,█▄▃▂▁
batch loss,0.28999
loss,6.9134


wandb: Agent Starting Run: pxc0sgjc with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.4
wandb: 	epochs: 10
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam


wandb: Ctrl + C detected. Stopping sweep.


打开输出结果的链接即可查看到各种超参数设置下模型的训练结果。